In [1]:
import cv2
import numpy as np
from datetime import datetime
import time #for FPS control
import re

In [2]:
#import pytesseract as tess

In [3]:
import easyocr
OCR = easyocr.Reader(['en','en'], gpu = True)

In [4]:
# Regex Conditioning

PalphaNum1 = re.compile(r'^[ABCDFJKMNPRTVW][^IOZ]{0,2}[0-9]{1,4}[^IOZ]{0,1}')               #General With Ended Suffix
PalphaNum3 = re.compile(r'^[Q][AKBCLMPRST][^QS]{0,2}[0-9]{1,4}[^QSIOZ]{0,1}')               #Sarawak with Ended Suffix
PalphaNum5 = re.compile(r'^[S][ABDGKLSTU][AB]{0,2}[^IOZQS]{0,2}[0-9]{1,4}[^QSIOZ]{0,1}')    #Sabah with End Suffix
PalphaNum7 = re.compile(r'^[H][B][0-9]{1,4}[S][A]')                                         #Shah Alam Taxi 
PalphaNum8 = re.compile(r'^[H][ABCDEJKLMNPQRSTW][A-Z][0-9]{1,4}')                            #Taxi General
PalphaNum9 = re.compile(r'^[L][I][M][O][0-9]{1,4}[WB]{0,2}')                                #Airport LIMO
PalphaNum10 = re.compile(r'^[Z][ADLUZ]{0,2}[0-9]{1,4}')                                      #Military Plates

#Diplomatic Corps, Consular Corps, UN numberp plates & Commemorative Plates are not included

In [5]:
# Removing Strings with Spcl Characters

SCregex = re.compile('[a-z@_!#$%^&*.,()''<>?/\|}{~:]')

In [6]:
# Defining Scale and Resolution

def make_1080p():
    cap.set(3, 1920)
    cap.set(4, 1080)

def make_720p():
    cap.set(3, 1280)
    cap.set(4, 720)

def make_480p():
    cap.set(3, 640)
    cap.set(4, 480)

def change_res(width, height):
    cap.set(3, width)
    cap.set(4, height)

In [7]:
# Upscale / DownScale

def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

### Number Plate Detection

In [10]:
plateCascade = cv2.CascadeClassifier("haarcascade_number_plate_recognition.xml")
minArea = 500

#cap = cv2.VideoCapture(1) # 0,1,2......for external camera sources
#cap = cv2.VideoCapture("Gate3.mp4")

frameWidth = 640    #Frame Width
franeHeight = 480   # Frame Height
cap.set(3,frameWidth)
cap.set(4,franeHeight)
cap.set(5,100)
#count = 0


#cap.set(cv2.CAP_PROP_FPS, 60)
#cap.set(cv2.CAP_PROP_AUTOFOCUS, 0) # turn the autofocus off 
#cap.set(cv2.CAP_PROP_FOCUS,10)
#cap.set(cv2.CAP_PROP_EXPOSURE,25)

if (cap.isOpened()== False):
    print("Can not open camera")

cdb = {}

#fpsLimit = 0 # throttle limit for FPS
startTime = time.time()

#make_1080p()
#cap.set(cv2.CAP_PROP_FPS, 30)

while True:
#while(cap.isOpened()):
    
    success , img  = cap.read()
    img = cv2.resize(img, None, None, fx=0.9, fy=0.9)
    #nowTime = time.time()
    #if (int(nowTime - startTime)) > fpsLimit:
        
    try:
    #if success == True:

        cv2.imshow('frame Detection', img)
        
        imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        numberPlates = plateCascade.detectMultiScale(imgGray, 1.5, 3)

        #############
        if len(numberPlates) > 0:
        #############
            for (x, y, w, h) in numberPlates:
                area = w*h
                if area > minArea:
                    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    #cv2.putText(img,"NumberPlate",(x,y-5),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
                    imgRoi = img[y:y+h,x:x+w]

                    ###############
                    output = OCR.readtext(imgRoi)
                    if len(output) != 0:
                        Text = output[0][1].replace(' ','')
                        #Conf = output[0][2]
                        #if Conf > 0.6:

                        if Text not in cdb.keys():
                            if(SCregex.search(Text) == None):
                                if PalphaNum1.search(Text) is not None:
                                    now = str(datetime.now())
                                    cdb[Text] = now
                                    cv2.imshow('Plate Detection', imgRoi)
                                elif PalphaNum3.search(Text) is not None:
                                    now = str(datetime.now())
                                    cdb[Text] = now
                                    cv2.imshow('Plate Detection', imgRoi)
                                elif PalphaNum5.search(text) is not None:
                                    now = str(datetime.now())
                                    cdb[text] = now
                                    cv2.imshow('Plate Detection', imgRoi)
                                elif PalphaNum7.search(Text) is not None:
                                    now = str(datetime.now())
                                    cdb[Text] = now
                                    cv2.imshow('Plate Detection', imgRoi)
                                elif PalphaNum8.search(Text) is not None:
                                    now = str(datetime.now())
                                    cdb[Text] = now
                                    cv2.imshow('Plate Detection', imgRoi)
                                elif PalphaNum9.search(Text) is not None:
                                    now = str(datetime.now())
                                    cdb[Text] = now
                                    cv2.imshow('Plate Detection', imgRoi)
                                elif PalphaNum10.search(Text) is not None:
                                    now = str(datetime.now())
                                    cdb[Text] = now
                                    cv2.imshow('Plate Detection', imgRoi)
                                else:
                                    pass


                    ################

                    #cv2.imshow("ROI",imgRoi)
                    cv2.putText(img,Text,(x,y-5),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)


            #cv2.imshow("Result",imgRoi)

        #if cv2.waitKey(1) & 0xFF ==ord('s'):
        #    cv2.imwrite(r"C:\Users\royku\Sem II Modules\Project Prep\RA\Living Lab Grant\HaarcascadeOutImages\Image"+str(count)+".jpg",imgRoi)
        #    cv2.rectangle(img,(0,200),(640,300),(0,255,0),cv2.FILLED)
        #    cv2.putText(img,"Scan Saved",(15,265),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
        #    cv2.imshow("Result",img)
        #    cv2.waitKey(500)
        #    count+=1
        startTime = time.time()
    except Exception as e:
        pass

    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [9]:
cdb

{'WRB2256': '2022-01-05 13:45:45.833813',
 'VRB2256': '2022-01-05 13:47:37.561558',
 'VDN5003': '2022-01-05 13:51:37.555560',
 'WUQ6390': '2022-01-05 13:53:35.030435',
 'VGC6842': '2022-01-05 13:53:41.824126',
 'KGC6842': '2022-01-05 13:53:42.228832',
 'ALY9972': '2022-01-05 13:55:16.517735',
 'ALJ9973': '2022-01-05 13:55:16.689280',
 'ALJ9972': '2022-01-05 13:55:17.205469',
 'ALY9973': '2022-01-05 13:55:29.776357',
 'VGD6121': '2022-01-05 13:56:01.527580'}

In [9]:
cdb

{}

### Car Number Plates with Time of Entry

In [21]:
cdb

{'BNW6192': '2021-12-02 12:10:55.375420',
 'VDY0918': '2021-12-02 12:11:11.584045',
 'VDY8918': '2021-12-02 12:11:12.597841',
 'VDY8910': '2021-12-02 12:11:14.565164',
 'VAW7978': '2021-12-02 12:11:42.735113',
 'JUL3344': '2021-12-02 12:12:05.036081',
 'VGJ82': '2021-12-02 12:12:19.325916',
 'AJP2551': '2021-12-02 12:12:26.686662',
 'VAH5268': '2021-12-02 12:12:29.620808'}

### Total Number of Cars

In [22]:
len(cdb)

9

In [7]:
cdb

{'BNW6192': '2021-11-30 13:07:10.353025',
 'VDY8310': '2021-11-30 13:07:25.491483',
 'VDY0310': '2021-11-30 13:07:25.522401',
 'VDY898': '2021-11-30 13:07:25.872207',
 'VDY8918': '2021-11-30 13:07:26.333678',
 'VDY8910': '2021-11-30 13:07:26.610936',
 'VDY0918': '2021-11-30 13:07:30.544293',
 'VAW7978': '2021-11-30 13:08:01.581541',
 'JUL3344': '2021-11-30 13:08:25.218096',
 'VGJ82': '2021-11-30 13:08:41.605677',
 'AJP2551': '2021-11-30 13:08:44.446922',
 'VAH5265': '2021-11-30 13:08:51.672181',
 'VAH5268': '2021-11-30 13:08:51.924507'}

In [22]:
cdb

{}

In [29]:
cdb

{'ND6493D': '2021-05-20 14:30:15.867461'}

In [28]:
cdb

{'HD 6493D': '2021-05-17 19:41:19.725860',
 'D 6493 D': '2021-05-17 19:41:20.254933'}

0